# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [47]:
%pip install iso3166

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [48]:
%pip install --upgrade plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import Statements

In [49]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [50]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [51]:
df_data = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [52]:
# Number of Rows and Columns, tuple
print(df_data.shape)
df_data.head()

(4324, 9)


,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [53]:
print(df_data.isnull().values.any())

# Find column which has NaN values
print("Columns with NaN values:")
for col in df_data.columns:
  if df_data[col].isnull().values.any():
    print(f"{col}")

print("Duplicates:")
df_data.duplicated().values.any()

True
Columns with NaN values:
Price
Duplicates:


False

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [54]:
# Removing columns containing NaN values
df_data_clean = df_data.dropna().reset_index(drop=True)
df_data_clean

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
3,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success
4,5,5,CASC,"LC-9, Taiyuan Satellite Launch Center, China","Sat Jul 25, 2020 03:13 UTC","Long March 4B | Ziyuan-3 03, Apocalypse-10 & N...",StatusActive,64.68,Success
...,...,...,...,...,...,...,...,...,...
959,3855,3855,US Air Force,"SLC-4W, Vandenberg AFB, California, USA","Fri Jul 29, 1966 18:43 UTC",Titan IIIB | KH-8,StatusRetired,59.0,Success
960,3971,3971,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA","Thu May 06, 1965 15:00 UTC",Titan IIIA | LES 2 & LCS 1,StatusRetired,63.23,Success
961,3993,3993,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA","Thu Feb 11, 1965 15:19 UTC",Titan IIIA | LES 1,StatusRetired,63.23,Success
962,4000,4000,US Air Force,"SLC-20, Cape Canaveral AFS, Florida, USA","Thu Dec 10, 1964 16:52 UTC",Titan IIIA | Transtage 2,StatusRetired,63.23,Success


## Descriptive Statistics

# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [55]:
# value_counts() returns Series
num_launches = df_data['Organisation'].value_counts()

# Series to Dataframe with 2 columns, reset index column and put previous index column in df and rename it 'Number_of_Launches'
num_launch_byorg = num_launches.rename_axis('Organisation').reset_index(name='Number_of_Launches')

# Bar plot
fig = px.bar(num_launch_byorg,x='Organisation', y='Number_of_Launches', color='Number_of_Launches')
fig.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [56]:
df_data['Rocket_Status'].value_counts()

StatusRetired    3534
StatusActive      790
Name: Rocket_Status, dtype: int64

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [57]:
df_data['Mission_Status'].value_counts()

Success              3879
Failure               339
Partial Failure       102
Prelaunch Failure       4
Name: Mission_Status, dtype: int64

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [58]:
# Column 'Price', remove comma from string and change data type to float, in order to plot histogram
df_data_clean['Price'] = df_data_clean['Price'].apply(lambda x: x.replace(',', ''))
df_data_clean['Price'] = df_data_clean['Price'].astype('float')

In [59]:
# Plot histogram
fig = px.histogram(df_data_clean, x='Price')
fig.show()

In [60]:
print(df_data_clean['Price'].max())
df_data_clean['Price'].min()


5000.0


5.3

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [61]:
# Extract Country from string in 'Location' column, take the last element after split, apply() helps to apply a lambda function to every row of dataframe
df_data['Country'] = df_data.apply(lambda row: row.Location.split(", ")[-1], axis=1)

In [62]:
# Change old Country names to up to date names
df_data.replace(to_replace='Russia', value='Russian Federation', inplace=True)
df_data.replace(to_replace='New Mexico', value='USA', inplace=True)
df_data.replace(to_replace='Yellow Sea', value='China', inplace=True)
df_data.replace(to_replace='Shahrud Missile Test Site', value='Iran', inplace=True)
df_data.replace(to_replace='Pacific Missile Range Facility', value='USA', inplace=True)
df_data.replace(to_replace='Barents Sea', value='Russian Federation', inplace=True)
df_data.replace(to_replace='Gran Canaria', value='USA', inplace=True)
df_data.replace(to_replace='Iran', value='Iran, Islamic Republic of', inplace=True)
df_data.replace(to_replace='North Korea', value="Korea, Democratic People's Republic of", inplace=True)
df_data.replace(to_replace='South Korea', value="Korea, Republic of", inplace=True)
df_data.replace(to_replace='Pacific Ocean', value="Liberia", inplace=True)

In [63]:
# Use class countries from iso3166 module, to get 3 letter name of Country
df_data['Country'] = df_data.apply(lambda row: countries.get(row.Country).alpha3, axis=1)

In [64]:
# module iso3166, printed all countries to see up to date names, which were used in replacement above
for c in countries:
  print(c)

Country(name='Afghanistan', alpha2='AF', alpha3='AFG', numeric='004', apolitical_name='Afghanistan')
Country(name='Åland Islands', alpha2='AX', alpha3='ALA', numeric='248', apolitical_name='Åland Islands')
Country(name='Albania', alpha2='AL', alpha3='ALB', numeric='008', apolitical_name='Albania')
Country(name='Algeria', alpha2='DZ', alpha3='DZA', numeric='012', apolitical_name='Algeria')
Country(name='American Samoa', alpha2='AS', alpha3='ASM', numeric='016', apolitical_name='American Samoa')
Country(name='Andorra', alpha2='AD', alpha3='AND', numeric='020', apolitical_name='Andorra')
Country(name='Angola', alpha2='AO', alpha3='AGO', numeric='024', apolitical_name='Angola')
Country(name='Anguilla', alpha2='AI', alpha3='AIA', numeric='660', apolitical_name='Anguilla')
Country(name='Antarctica', alpha2='AQ', alpha3='ATA', numeric='010', apolitical_name='Antarctica')
Country(name='Antigua and Barbuda', alpha2='AG', alpha3='ATG', numeric='028', apolitical_name='Antigua and Barbuda')
Countr

In [65]:
df_Launches_by_C = df_data['Country'].value_counts().reset_index()
df_Launches_by_C.columns = ['Country', 'Total number of launches']
df_Launches_by_C['Country Name'] = df_Launches_by_C.apply(lambda row: countries.get(row.Country).name, axis=1)
df_Launches_by_C

,Country,Total number of launches,Country Name
0,RUS,1398,Russian Federation
1,USA,1351,United States of America
2,KAZ,701,Kazakhstan
3,FRA,303,France
4,CHN,269,China
5,JPN,126,Japan
6,IND,76,India
7,LBR,36,Liberia
8,IRN,14,"Iran, Islamic Republic of"
9,NZL,13,New Zealand


In [95]:
#Plot Choropleth
fig = px.choropleth(df_Launches_by_C, locations='Country', color='Total number of launches', hover_name='Country Name', color_continuous_scale=px.colors.sequential.matter)
fig.show()

# Use a Choropleth Map to Show the Number of Failures by Country


In [67]:
df_data.loc[df_data.Mission_Status == 'Failure', 'Mission_Status'] = 'Failure'
df_data.loc[df_data.Mission_Status == 'Prelaunch Failure', 'Mission_Status'] = 'Failure'
df_data.loc[df_data.Mission_Status == 'Partial Failure', 'Mission_Status'] = 'Failure'
df_data.loc[df_data.Mission_Status == 'Success', 'Mission_Status'] = 'Success'

df_data_SF = df_data.groupby(['Country', 'Mission_Status']).count().reset_index()

In [68]:
df_Failures_by_C = df_data_SF.loc[df_data_SF['Mission_Status'] == 'Failure']
df_Failures_by_C = df_Failures_by_C.rename(columns={'Rocket_Status' : 'Number_of_Failures'})
df_Failures_by_C.drop(columns=['Mission_Status', 'Unnamed: 0', 'Unnamed: 0.1', 'Organisation', 'Location', 'Date', 'Detail', 'Price'], inplace=True)
df_Failures_by_C['Country Name'] = df_Failures_by_C.apply(lambda row: countries.get(row.Country).name, axis=1)

In [94]:
fig = px.choropleth(df_Failures_by_C, locations='Country', color='Number_of_Failures', hover_name='Country Name', color_continuous_scale=px.colors.sequential.matter)
fig.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [70]:
fig = px.sunburst(df_data, path=['Country', 'Organisation', 'Mission_Status'], maxdepth=2, width=1000, height=800, color_discrete_sequence=px.colors.qualitative.Pastel)
fig.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [71]:
# Use df_data_clean, NaN values in 'Price' column are droped
df_total_amount = df_data_clean.groupby(by='Organisation')['Price'].sum().reset_index()
df_total_amount['Number_of_Launches'] = df_total_amount.apply(lambda row: df_data_clean['Organisation'].value_counts()[row['Organisation']], axis=1)
df_total_amount.sort_values(by=['Price'], ascending=False)


,Organisation,Price,Number_of_Launches
14,NASA,"76,280.00",149
0,Arianespace,"16,345.00",96
21,ULA,"14,798.00",98
16,RVSN USSR,"10,000.00",2
2,CASC,"6,340.26",158
20,SpaceX,"5,444.00",99
15,Northrop,"3,930.00",83
12,MHI,"3,532.50",37
8,ISRO,"2,177.00",67
22,US Air Force,"1,550.92",26


# Analyse the Amount of Money Spent by Organisation per Launch

In [72]:
df_total_amount['Money_per_Launch_by_Org'] = df_total_amount['Price'] / df_total_amount['Number_of_Launches']
df_total_amount.sort_values(by=['Money_per_Launch_by_Org'], ascending=False)

,Organisation,Price,Number_of_Launches,Money_per_Launch_by_Org
16,RVSN USSR,"10,000.00",2,"5,000.00"
14,NASA,"76,280.00",149,511.95
1,Boeing,"1,241.00",7,177.29
0,Arianespace,"16,345.00",96,170.26
21,ULA,"14,798.00",98,151.00
7,ILS,"1,320.00",13,101.54
12,MHI,"3,532.50",37,95.47
13,Martin Marietta,721.40,9,80.16
22,US Air Force,"1,550.92",26,59.65
9,JAXA,168.00,3,56.00


# Chart the Number of Launches per Year

In [73]:
df_data.head()
df_data['Year'] = df_data['Date'].apply(lambda x: x.split(', ')[1][0:4])
df_data['Launches_per_Year'] = df_data.groupby('Year')['Year'].transform('count')

df_data_launch_per_year = df_data[['Year', 'Launches_per_Year']].copy()
df_data_launch_per_year.drop_duplicates(inplace=True)

fig = px.bar(df_data_launch_per_year.sort_values(by='Year'), x='Year', y='Launches_per_Year')
fig.show()

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

In [74]:
df_data.head()

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Country,Year,Launches_per_Year
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,USA,2020,63
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,CHN,2020,63
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,USA,2020,63
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,KAZ,2020,63
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,USA,2020,63


In [75]:
# pandas.to_datetime converts a scalar, array-like, Series or DataFrame/dict-like to a pandas datetime object
df_data['Date'] = pd.to_datetime(df_data['Date'], utc=True)
df_data['month'] = df_data['Date'].dt.month_name()

df_data['Launches_per_Month'] = df_data.groupby('month')['month'].transform('count')

df_data_launches_per_month = df_data[['month', 'Launches_per_Month']].copy()
df_data_launches_per_month.drop_duplicates(inplace=True)

month_dict = {'January':1,'February':2,'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
df_data_launches_per_month.sort_values('month', key = lambda x : x.apply (lambda x : month_dict[x]), inplace=True)



fig = px.bar(df_data_launches_per_month, x='month', y='Launches_per_Month', color='Launches_per_Month')
fig.show()


# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

In [76]:
df_data_clean['Year'] = df_data_clean['Date'].apply(lambda x: x.split(', ')[1][0:4])

In [77]:
df_price_by_year = df_data_clean.groupby(by="Year")["Price"].sum().reset_index()
df_price_by_year['Number_of_Launches'] = df_price_by_year.apply(lambda row: df_data_clean['Year'].value_counts()[row['Year']], axis=1)

fig = px.bar(df_price_by_year, x='Year', y='Number_of_Launches', color='Number_of_Launches', title='Number of launches over time (Note: Launches without price NOT INCLUDED)')
fig.show()

In [78]:
df_price_by_year['price_per_launch'] = df_price_by_year['Price'] / df_price_by_year['Number_of_Launches']
fig = px.line(df_price_by_year, x="Year", y="price_per_launch", title='Average price of rocket launches over time')
fig.show()

In [79]:
df_price_by_year

,Year,Price,Number_of_Launches,price_per_launch
0,1964,126.46,2,63.23
1,1965,126.46,2,63.23
2,1966,177.00,3,59.00
3,1967,"1,573.00",8,196.62
4,1968,"2,792.00",10,279.20
5,1969,"4,876.00",8,609.50
6,1970,"1,160.00",1,"1,160.00"
7,1971,"2,320.00",2,"1,160.00"
8,1972,"2,320.00",2,"1,160.00"
9,1973,"1,160.00",1,"1,160.00"


# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

In [80]:
grouped = df_data.groupby(by=['Organisation', 'Year']).count().reset_index()
grouped.rename(columns={'Country' : 'Number_of_Launches'}, inplace=True)
launches_over_time_per_org = grouped[['Organisation', 'Year', 'Number_of_Launches']]

# use previously created df, to get list of top10 organizations
top10_org = num_launch_byorg.head(10)
top10_org_list = top10_org['Organisation'].tolist()

  
# select rows if org is in top10 list
rslt_df = launches_over_time_per_org.loc[launches_over_time_per_org['Organisation'].isin(top10_org_list)]
rslt_df['Year'] = rslt_df['Year'].astype(float)



fig = px.line(rslt_df.sort_values(by=['Year']), x="Year", y="Number_of_Launches", color='Organisation')
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

In [81]:
# Filter data up until 1991
df_data['Year'] = df_data.Year.astype(int)
df_data_filtered = df_data[df_data['Year'] <= 1991]

# Drop unnecessary columns
df_data_filtered.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Launches_per_Year', 'Launches_per_Month'], inplace=True)

# Take into account former Soviet Republic Kazakhstan --> USSR
df_data_filtered.loc[df_data_filtered['Country'] == 'RUS', 'Country'] = 'USSR'
df_data_filtered.loc[df_data_filtered['Country'] == 'KAZ', 'Country'] = 'USSR'

# Filter countries USA and USSR
df_data_filtered_v2 = df_data_filtered.loc[df_data_filtered['Country'].isin(['USA', 'USSR'])]
df_data_filtered_v2

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Country,Year,month
1718,RVSN USSR,"Site 175/58, Baikonur Cosmodrome, Kazakhstan",1991-12-20 00:00:00+00:00,Rokot/Briz K | GVM Demo Flight,StatusRetired,NaN,Success,USSR,1991,December
1719,Yuzhmash,"Site 32/2, Plesetsk Cosmodrome, Russia",1991-12-18 03:54:00+00:00,Tsyklon-3 | Intercosmos 25 & Magion 3,StatusRetired,NaN,Success,USSR,1991,December
1721,General Dynamics,"SLC-36B, Cape Canaveral AFS, Florida, USA",1991-12-07 22:47:00+00:00,Atlas II | Eutelsat 2F3,StatusRetired,NaN,Success,USA,1991,December
1722,General Dynamics,"SLC-3W, Vandenberg AFB, California, USA",1991-11-28 13:23:00+00:00,Atlas-E/F Star-37S-ISS | DMSP F-11,StatusRetired,NaN,Success,USA,1991,November
1723,RVSN USSR,"Site 133/3, Plesetsk Cosmodrome, Russia",1991-11-27 03:30:00+00:00,Cosmos-3M (11K65M) | Cosmos 2173,StatusRetired,NaN,Success,USSR,1991,November
...,...,...,...,...,...,...,...,...,...,...
4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05 07:33:00+00:00,Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure,USA,1958,February
4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01 03:48:00+00:00,Juno I | Explorer 1,StatusRetired,NaN,Success,USA,1958,February
4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06 16:44:00+00:00,Vanguard | Vanguard TV3,StatusRetired,NaN,Failure,USA,1957,December
4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03 02:30:00+00:00,Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success,USSR,1957,November


## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

In [82]:
df_USSR_vs_USA = df_data_filtered_v2['Country'].value_counts().rename_axis('Country').reset_index(name='Number_of_Launches')

fig = px.pie(df_USSR_vs_USA, values='Number_of_Launches', names='Country', title='Number of Launches USA vs USSR')
fig.show()

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

In [83]:
df_data_filtered_v3 = df_data_filtered_v2[['Country', 'Year']]

num_launches_l = df_data_filtered_v3.groupby(['Country','Year'])['Year'].count().tolist()

df_sorted = df_data_filtered_v3.sort_values(by=['Country', 'Year'])
df_sorted.drop_duplicates(inplace=True)

df_sorted['Num_of_Launches'] = num_launches_l

df_sorted

fig = px.line(df_sorted, x="Year", y="Num_of_Launches", color='Country')
fig.show()

## Chart the Total Number of Mission Failures Year on Year.

In [84]:
df_mission_failures = df_data_filtered_v2[df_data_filtered_v2['Mission_Status'] == 'Failure']

num_failures = df_mission_failures.groupby(['Country','Year'])['Year'].count().tolist()

df_sorted = df_mission_failures.sort_values(by=['Country', 'Year'])
df_sorted.drop_duplicates(subset=['Country', 'Year'], inplace=True)

df_sorted['Num_of_Failures'] = num_failures

In [85]:
fig = px.line(df_sorted, x="Year", y="Num_of_Failures", color='Country')
fig.show()

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

In [86]:
total_num_failures = df_sorted.groupby(by=['Country', 'Year'])['Num_of_Failures'].sum().tolist()
total_num_failures_USA = sum(total_num_failures[:(len(total_num_failures)//2)])
total_num_failures_USSR = sum(total_num_failures[(len(total_num_failures)//2):])

def f(row):
    if row['Country'] == 'USA':
        val = row['Num_of_Failures']/total_num_failures_USA*100
    else:
        val = row['Num_of_Failures']/total_num_failures_USSR*100
    return val

df_sorted['Precentage_of_Failures'] = round(df_sorted.apply(f, axis=1)).astype(int)

In [87]:
fig = px.line(df_sorted, x="Year", y="Precentage_of_Failures", color='Country')
fig.show()

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

In [88]:
# value_counts() returns Series
num_launches_per_Country = df_data[['Year','Country']].value_counts().reset_index(name='Number_Launches_by_Country')
num_launches_per_Country.sort_values(by='Year', inplace=True)

# Sort and remove duplicates (in this case we will keep only one year)
lead_country_by_launches = num_launches_per_Country.sort_values('Number_Launches_by_Country', ascending=False).drop_duplicates(['Year'])
lead_country_by_launches.sort_values(by='Year', inplace=True)

# Bar plot
fig = px.bar(num_launches_per_Country,x='Year', y='Number_Launches_by_Country', color='Country', title='Number of Launches by Country per Year')
fig.show()

# Bar plot Lead Country by launches per year
fig2 = px.bar(lead_country_by_launches,x='Year', y='Number_Launches_by_Country', color='Country', title='Lead Country by Launches per Year')
fig2.show()

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 

In [93]:
# value_counts() returns Series
num_launches_per_Organisation = df_data[['Year','Organisation']].value_counts().reset_index(name='Number_Launches_by_Organisation')
num_launches_per_Organisation.sort_values(by='Year', inplace=True)

# Sort and remove duplicates (in this case we will keep only one year)
lead_org_by_launches = num_launches_per_Organisation.sort_values('Number_Launches_by_Organisation', ascending=False).drop_duplicates(['Year'])
lead_org_by_launches.sort_values(by='Year', inplace=True)

# Bar plot
fig = px.bar(lead_org_by_launches,x='Year', y='Number_Launches_by_Organisation', color='Organisation', title='Organisation with max Launches per Year')
fig.show()

